In [1]:
import os, sys
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
    
import os
from model.pipeline import build_feature_matrix_and_target
from data_processing.split import split_member_time
from data_processing.grid_aggregate import conus_mean
from model.ednn_model import ednn_regressor_model
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from model.load_ednn_model import load_ednn_model, set_training_variance, predict_ednn
from mlguess.keras.models import EvidentialRegressorDNN
from mlguess.keras.losses import evidential_reg_loss
import xarray as xr

2025-08-04 21:26:38.415423: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-04 21:26:38.479487: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-04 21:26:39.586897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#---------------- Build the full feature & target arrays
X, y = build_feature_matrix_and_target()

#---------------- Split & scale (returns 8 objects)
(
    X_train_xr, X_val_xr,
    y_train_xr, y_val_xr,
    X_train,    X_val,
    y_train,    y_val
) = split_member_time(X, y)

#---------------- Sanity‐check shapes
print("Full X, y shapes:         ", X.shape, y.shape)
print("X_train_xr, X_val_xr:     ", X_train_xr.shape, X_val_xr.shape)
print("X_train, X_val:           ", X_train.shape, X_val.shape)
print("y_train_xr, y_val_xr:     ", y_train_xr.shape, y_val_xr.shape)
print("y_train, y_val:           ", y_train.shape, y_val.shape)

In [ ]:
model_folder = "/bsuhome/ksilwimba/scratch/clm5-lai-ppe-ednn/temporal_ppe_emulation/saved_model"
model_path   = f"{model_folder}/ednn_reg_model.keras"
loss_weight  = 1e-10#1e-10   # match what you trained with

model = load_ednn_model(model_path, loss_weight)
set_training_variance(model, y_train)
p_wu_val, p_wo_val = predict_ednn(model, X_val)

In [ ]:
#---------------- Wrap train predictions back to xarray and unstack
param_names = ["mu","nu","alpha","beta"]

#---------------- Wrap validation predictions back to xarray and unstack
pred_val_ds = xr.Dataset(
    { name: (("sample",), p_wo_val[:, i]) 
      for i, name in enumerate(param_names) },
    coords={"sample": y_val_xr.coords["sample"]}
)

pred_val_ds["TLAI_unc"] = xr.DataArray(
    p_wu_val,
    dims=("sample","uncertainty"),
    coords={
      "sample":      y_val_xr.coords["sample"],
      "uncertainty": ["mean","aleatoric","epistemic"]
    },
    name="TLAI_uncertainties"
)

pred_val_3d = pred_val_ds.unstack("sample")

#---------------- Inspect
print("VALIDATION PREDICTIONS:", pred_val_3d)

In [ ]:
lai_glob = "/bsuhome/ksilwimba/scratch/NCAR/Data/LAI/TLAI/*.nc"
land_area_path = "/bsuhome/ksilwimba/scratch/NCAR/Data/helpers/sparsegrid_landarea.nc"

#------------- Load land area
land_area = xr.open_dataset(land_area_path).landarea

In [ ]:
y_val_3d = y_val_xr.unstack("sample")
pred_val_3d_ = conus_mean(pred_val_3d, land_area)
y_val_3d_ = conus_mean(y_val_3d, land_area)